In [26]:
!pip install -U -q PyDrive
!pip install zipfile
!pip install gdown

ERROR: Could not find a version that satisfies the requirement zipfile (from versions: none)
ERROR: No matching distribution found for zipfile


In [27]:
!gdown --id 1lU0YqwpIaSKeUZAXzs9RwTYx8gsw9685

/opt/conda/lib/python3.10/site-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1lU0YqwpIaSKeUZAXzs9RwTYx8gsw9685
From (redirected): https://drive.google.com/uc?id=1lU0YqwpIaSKeUZAXzs9RwTYx8gsw9685&confirm=t&uuid=8afafa2f-3f94-460e-b96c-d192bd17cce9
To: /kaggle/working/SummaryFlow_Run4.zip
100%|██████████████████████████████████████| 1.63G/1.63G [00:17<00:00, 95.4MB/s]


In [28]:
import sys
sys.path.append('/kaggle/input/wholenovels')
import os
import zipfile
from BookSpliter2 import split_book
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

# Path to the ZIP file containing the model
model_zip_path = '/kaggle/working/SummaryFlow_Run4.zip'

# Create the directory to extract the model
os.makedirs('/kaggle/working/model', exist_ok=True)

# Extract the model from the ZIP file
with zipfile.ZipFile(model_zip_path, 'r') as zip_ref:
    zip_ref.extractall('/kaggle/working/model')

2024-04-14 19:59:38.901300: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-14 19:59:38.901408: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-14 19:59:39.171349: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [29]:
import torch

# Check if CUDA is available and set PyTorch to use GPU
if torch.cuda.is_available(): 
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 2 GPU(s) available.
We will use the GPU: Tesla T4


In [30]:
# Load Pegasus Model & Tokenizer
model_checkpoint = '/kaggle/working/model'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint).to(device)

In [31]:
# Summarize the book recursively
def summarize_book(book):
    # Split the book into paragraphs
    splited_book = split_book(book)
    if len(splited_book) <= 5: # Base case
        print('\nThe Book Has Been Summarized Successfully!!\n')
        result_book = "\n\n".join(splited_book)
        return result_book
    else:
        print('\n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print('\nThe book needs a round of summarization.\n')
        print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!\n')
        summary_list = []
        for i in range(len(splited_book)): # Loop through all paragraphs to generate summaries
            # Tokenize the input text
            inputs = tokenizer(splited_book[i], return_tensors="pt", max_length=1024, truncation=True)
            # Generate summary
            summary_ids = model.generate(inputs.input_ids.to(model.device))
            # Decode the summary tokens
            summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
            # Add to summary list
            summary_list.append(summary)
            #print('\n--The original paragraph : \n')
            #print(splited_book[i])
            #print('\n--The paragraph summary : \n')
            #print(summary)
        # Combine all summarise into a single paragraph
        summarized_book = "\n".join(summary_list)
        # Recursively summarize the generated book
        print('\nA level is summarized successfully!!\n')
        #print('Summarized book : ')
        #print(summarized_book)
        return summarize_book(summarized_book)


In [ ]:
# Try to summarize a book

# Take the book path
book_path = r"/kaggle/input/wholenovels/animal_farm.txt"

# Read the book from the pdf file
with open(book_path, 'r') as file:
    # Read the entire contents of the file into a string
    book_content = file.read()

# Summarize the book and print it 
final_summary = summarize_book(book_content)
print("************************************************************************\n")
print("Final Book Summary : \n")
print(final_summary)
print("\n************************************************************************")

In [ ]:
# BertScore for the generated summary
from bert_score import score
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

generated_summary = r'''
'''
reference_summary = r'''
'''

generated_tokens = tokenizer.tokenize(generated_summary)
reference_tokens = tokenizer.tokenize(reference_summary)

# Join tokens into a single string
generated_string = ' '.join(generated_tokens)
reference_string = ' '.join(reference_tokens)

P, R, F1 = score([generated_string], [reference_string], lang='en', verbose=True)

print("Precision:", P.item())
print("Recall:", R.item())
print("F1 Score:", F1.item())
